In [1]:
import pandas as pd
import numpy as np

In [2]:
num_players = 5
deck_df = pd.read_csv('deck.csv')
hand_rankings_df = pd.read_csv('poker_hand_rankings.csv')
card_rankings_df = pd.read_csv('card_rankings.csv')
# Initialize empty dictionary to store player hands
player_hands = {i: [] for i in range(num_players)}
# Initialize empty dictionary to store table cards (5 slots)
table_cards = {i: [] for i in range(5)}



In [ ]:
table_cards

In [ ]:
player_hands

In [3]:
def deck_shuffler(deck_df):
    shuffled_df = deck_df.copy()
    shuffled_df = shuffled_df.iloc[np.random.permutation(len(shuffled_df))]
    return shuffled_df

def initial_deal(shuffled_df, player_hands):
    # Deal 2 cards to each player
    for card_num in range(2):  # Deal 2 rounds
        for player in player_hands.keys():  # Deal to each player
            # Get the next card (first row of shuffled deck)
            card = shuffled_df.iloc[player + (card_num * len(player_hands))]
            # Add card to player's hand as [suit, rank] list
            player_hands[player].append([card['suit'], card['rank']])
    
    # Remove dealt cards from the deck (2 cards * number of players)
    remaining_deck = shuffled_df.iloc[2 * len(player_hands):].reset_index(drop=True)
    
    return player_hands, remaining_deck


def burn_card(remaining_deck):
    # Burn one card from the remaining deck
    burned_card = remaining_deck.iloc[0]
    remaining_deck = remaining_deck.iloc[1:].reset_index(drop=True)
    return remaining_deck

def deal_flop(remaining_deck, table_cards):
    # Deal 3 cards to the table
    for i in range(3):
        card = remaining_deck.iloc[0]
        # Add card to table as [suit, rank] list, similar to player hands
        table_cards[i] = [card['suit'], card['rank']]
        remaining_deck = remaining_deck.iloc[1:].reset_index(drop=True)
    return table_cards, remaining_deck

def deal_turn(remaining_deck, table_cards):
    # Deal 1 card to the table in position 3
    card = remaining_deck.iloc[0]
    # Add card to table as [suit, rank] list
    table_cards[3] = [card['suit'], card['rank']]
    remaining_deck = remaining_deck.iloc[1:].reset_index(drop=True)
    return table_cards, remaining_deck

def deal_river(remaining_deck, table_cards):
    # Deal 1 card to the table in position 4
    card = remaining_deck.iloc[0]
    # Add card to table as [suit, rank] list
    table_cards[4] = [card['suit'], card['rank']]
    remaining_deck = remaining_deck.iloc[1:].reset_index(drop=True)
    return table_cards, remaining_deck

def hand_ranker(player_hands, table_cards):
    # For each player
    for player in player_hands.keys():
        # Combine player's hole cards with table cards
        all_cards = player_hands[player] + [card for card in table_cards.values() if card != []]
        
        # Create lists of just the ranks and suits
        ranks = [card[1] for card in all_cards]
        suits = [card[0] for card in all_cards]
        
        # Check for pairs by counting rank occurrences
        rank_counts = {}
        for rank in ranks:
            if rank in rank_counts:
                rank_counts[rank] += 1
            else:
                rank_counts[rank] = 1
        
        # Find any pairs (rank that appears exactly twice)
        pairs = [rank for rank, count in rank_counts.items() if count == 2]
        
        # Store results in player_hands dictionary and print for this player
        if pairs:
            player_hands[player] = {'cards': player_hands[player], 'hand_type': f"Pair of {pairs[0]}s"}
            print(f"Player {player} has a pair of {pairs[0]}s")
        else:
            player_hands[player] = {'cards': player_hands[player], 'hand_type': "No pair"}
            print(f"Player {player} has no pair")
            
    return player_hands



In [4]:
# Shuffle the deck
shuffled_df = deck_shuffler(deck_df)
player_hands, remaining_deck = initial_deal(shuffled_df, player_hands)
remaining_deck = burn_card(remaining_deck)
table_cards, remaining_deck = deal_flop(remaining_deck, table_cards)
remaining_deck = burn_card(remaining_deck)
table_cards, remaining_deck = deal_turn(remaining_deck, table_cards)
remaining_deck = burn_card(remaining_deck)
table_cards, remaining_deck = deal_river(remaining_deck, table_cards)
hand_rankings = hand_ranker(player_hands, table_cards)

Player 0 has a pair of 5s
Player 1 has no pair
Player 2 has no pair
Player 3 has a pair of 11s
Player 4 has a pair of 7s


In [ ]:
player_hands

In [ ]:
table_cards

In [ ]:
remaining_deck

In [ ]:
player_hands

In [ ]:
remaining_deck

In [ ]:
player_hands

In [ ]:
shuffled_df